In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab

We don't want to use scans that have no Majority Label nor Correct Label as these still need to be looked at by
neurosurgeons to verify if they have hemorrhaging or not

In [3]:
epi = pd.read_csv("Results_Epidural Hemorrhage Detection_2020-11-16_21.31.26.148.csv", error_bad_lines=False)
clean_epi = epi[(epi['Majority Label'] != '[]') | (epi['Correct Label'].notna()) & (epi['Correct Label'] != '[[], []]') ]

C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\3674797304.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  epi = pd.read_csv("Results_Epidural Hemorrhage Detection_2020-11-16_21.31.26.148.csv", error_bad_lines=False)


In [4]:
clean_epi.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1196514,ID_004c4b319.jpg,https://go.centaurlabs.com/problem/1196514,Labeled,"['[{""x"": 0.40818718, ""y"": 0.8623607}, {""x"": 0....",19.0,5.0,"[{""x"": 0.537109375, ""y"": 0.89453125}, {""x"": 0....",1.0,"[[{'x': 0.537109375, 'y': 0.89453125}, {'x': 0...",0.00,0.968851,NaN,NaN
1,1196682,ID_0237b835f.jpg,https://go.centaurlabs.com/problem/1196682,Ready,"['[{""x"": 0.62959146, ""y"": 0.720186}, {""x"": 0.6...",51.0,28.0,"[{""x"": 0.654296875, ""y"": 0.724609375}, {""x"": 0...",1.0,NaN,NaN,0.063519,NaN,NaN
2,1196477,ID_02e2d273d.jpg,https://go.centaurlabs.com/problem/1196477,Labeled,"['[{""x"": 0.35205495, ""y"": 0.34680602}, {""x"": 0...",22.0,6.0,"[{""x"": 0.46484375, ""y"": 0.357421875}, {""x"": 0....",1.0,"[[{'x': 0.46484375, 'y': 0.357421875}, {'x': 0...",0.06,0.710300,NaN,NaN
3,1196612,ID_02f37ce84.jpg,https://go.centaurlabs.com/problem/1196612,Ready,"['[{""x"": 0.3680166, ""y"": 0.48387372}, {""x"": 0....",51.0,29.0,"[{""x"": 0.431640625, ""y"": 0.607421875}, {""x"": 0...",2.0,NaN,NaN,0.264134,NaN,NaN
4,1196612,ID_02f37ce84.jpg,https://go.centaurlabs.com/problem/1196612,Ready,"['[{""x"": 0.3680166, ""y"": 0.48387372}, {""x"": 0....",51.0,29.0,"[{""x"": 0.353515625, ""y"": 0.59765625}, {""x"": 0....",2.0,NaN,NaN,0.264134,NaN,NaN


In [5]:
intra = pd.read_csv("Results_Intraparenchymal Hemorrhage Detection_2020-11-16_21.39.31.268.csv",error_bad_lines=False)
clean_intra = intra[(intra['Majority Label'] != '[]') | (intra['Correct Label'].notna()) & (intra['Correct Label'] != '[[]]') ]
#not sure what to do with scans that have "Gold Standard" (as their labeling state) since some of these have nans for both
#Majority Label and Correct Label
intra.shape
#original excel file has 1373 rows, not exactly sure why pd only reads 1289 even after deleting
#the "error_bad_lines=False" command

C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\3798969853.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  intra = pd.read_csv("Results_Intraparenchymal Hemorrhage Detection_2020-11-16_21.39.31.268.csv",error_bad_lines=False)


(1289, 14)

In [6]:
clean_intra.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1675660,ID_0002081b6.jpg,https://go.centaurlabs.com/problem/1675660,In Progress,"['[{""x"": 0.31658885, ""y"": 0.48357323}, {""x"": 0...",22.0,9.0,"[{""x"": 0.3046875, ""y"": 0.484375}, {""x"": 0.3046...",1.0,NaN,NaN,0.224542,NaN,NaN
1,1675942,ID_0048e60a4.jpg,https://go.centaurlabs.com/problem/1675942,In Progress,"['[{""x"": 0.54811704, ""y"": 0.5961946}, {""x"": 0....",22.0,7.0,"[{""x"": 0.5859375, ""y"": 0.640625}, {""x"": 0.5859...",1.0,NaN,NaN,0.772365,NaN,NaN
2,1197689,ID_0057d2736.jpg,https://go.centaurlabs.com/problem/1197689,Labeled,"['[{""x"": 0.3040109, ""y"": 0.52168596}, {""x"": 0....",46.0,9.0,"[{""x"": 0.3046875, ""y"": 0.521484375}, {""x"": 0.3...",1.0,"[[{'x': 0.3046875, 'y': 0.521484375}, {'x': 0....",0.02,0.738368,NaN,NaN
3,1197754,ID_007cac3b7.jpg,https://go.centaurlabs.com/problem/1197754,Labeled,"['[{""x"": 0.41792268, ""y"": 0.56897116}, {""x"": 0...",51.0,10.0,"[{""x"": 0.41015625, ""y"": 0.615234375}, {""x"": 0....",1.0,"[[{'x': 0.4140625, 'y': 0.6171875}, {'x': 0.41...",0.03,0.618086,NaN,NaN
4,1675775,ID_00b46c6bc.jpg,https://go.centaurlabs.com/problem/1675775,In Progress,"['[{""x"": 0.82215744, ""y"": 0.20116618}, {""x"": 0...",22.0,6.0,"[{""x"": 0.365234375, ""y"": 0.693359375}, {""x"": 0...",1.0,NaN,NaN,0.006388,NaN,NaN


In [7]:
multiple = pd.read_csv("Results_Multiple Hemorrhage Detection_2020-11-16_21.36.24.018.csv",error_bad_lines=False )
#again, excel file has more rows but pd reading less
clean_multiple = multiple[(multiple['Majority Label'] != '[]') | (multiple['Correct Label'].notna()) & (multiple['Correct Label'] != '[[]]') ]
multiple.shape

C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\2722983080.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  multiple = pd.read_csv("Results_Multiple Hemorrhage Detection_2020-11-16_21.36.24.018.csv",error_bad_lines=False )


(1054, 14)

In [8]:
clean_multiple.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1198534,ID_01af922c3.jpg,https://go.centaurlabs.com/problem/1198534,In Progress,"['[{""x"": 0.372072, ""y"": 0.6399058}, {""x"": 0.36...",23.0,6.0,"[{""x"": 0.396484375, ""y"": 0.73046875}, {""x"": 0....",3.0,NaN,NaN,0.790347,NaN,NaN
1,1198534,ID_01af922c3.jpg,https://go.centaurlabs.com/problem/1198534,In Progress,"['[{""x"": 0.372072, ""y"": 0.6399058}, {""x"": 0.36...",23.0,6.0,"[{""x"": 0.619140625, ""y"": 0.765625}, {""x"": 0.61...",3.0,NaN,NaN,0.790347,NaN,NaN
2,1198534,ID_01af922c3.jpg,https://go.centaurlabs.com/problem/1198534,In Progress,"['[{""x"": 0.372072, ""y"": 0.6399058}, {""x"": 0.36...",23.0,6.0,"[{""x"": 0.46875, ""y"": 0.607421875}, {""x"": 0.468...",3.0,NaN,NaN,0.790347,NaN,NaN
3,1198791,ID_01c457a64.jpg,https://go.centaurlabs.com/problem/1198791,Labeled,"['[{""x"": 0.40734115, ""y"": 0.5586468}, {""x"": 0....",34.0,6.0,"[{""x"": 0.4375, ""y"": 0.7109375}, {""x"": 0.4375, ...",3.0,"[[{'x': 0.548828125, 'y': 0.6796875}, {'x': 0....",0.48,0.124994,NaN,NaN
4,1198791,ID_01c457a64.jpg,https://go.centaurlabs.com/problem/1198791,Labeled,"['[{""x"": 0.40734115, ""y"": 0.5586468}, {""x"": 0....",34.0,6.0,"[{""x"": 0.546875, ""y"": 0.671875}, {""x"": 0.54687...",3.0,"[[{'x': 0.548828125, 'y': 0.6796875}, {'x': 0....",0.48,0.124994,NaN,NaN


In [9]:
subarachnoid = pd.read_csv("Results_Subarachnoid Hemorrhage Detection_2020-11-16_21.36.18.668.csv", error_bad_lines=False)
clean_subarachnoid = subarachnoid[(subarachnoid['Majority Label'] != '[]') | (subarachnoid['Correct Label'].notna()) & (subarachnoid['Correct Label'] != '[[]]') ]
subarachnoid.shape
#original excel file has 1054 rows, not exactly sure why pd only reads 1031 even after deleting
#the "error_bad_lines=False" command

C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\1871949402.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  subarachnoid = pd.read_csv("Results_Subarachnoid Hemorrhage Detection_2020-11-16_21.36.18.668.csv", error_bad_lines=False)


(1031, 14)

In [10]:
clean_subarachnoid.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1197112,ID_001b1b944.jpg,https://go.centaurlabs.com/problem/1197112,In Progress,"['[{""x"": 0.43056944, ""y"": 0.5936399}, {""x"": 0....",16.0,5.0,"[{""x"": 0.314453125, ""y"": 0.75}, {""x"": 0.314453...",3.0,NaN,NaN,0.252748,NaN,NaN
1,1197112,ID_001b1b944.jpg,https://go.centaurlabs.com/problem/1197112,In Progress,"['[{""x"": 0.43056944, ""y"": 0.5936399}, {""x"": 0....",16.0,5.0,"[{""x"": 0.400390625, ""y"": 0.796875}, {""x"": 0.40...",3.0,NaN,NaN,0.252748,NaN,NaN
2,1197112,ID_001b1b944.jpg,https://go.centaurlabs.com/problem/1197112,In Progress,"['[{""x"": 0.43056944, ""y"": 0.5936399}, {""x"": 0....",16.0,5.0,"[{""x"": 0.556640625, ""y"": 0.302734375}, {""x"": 0...",3.0,NaN,NaN,0.252748,NaN,NaN
3,1197367,ID_005f428d2.jpg,https://go.centaurlabs.com/problem/1197367,Labeled,"['[{""x"": 0.59604716, ""y"": 0.4975766}, {""x"": 0....",29.0,5.0,"[{""x"": 0.54296875, ""y"": 0.763671875}, {""x"": 0....",2.0,"[[{'x': 0.626953125, 'y': 0.30078125}, {'x': 0...",0.33,0.763632,NaN,NaN
4,1197367,ID_005f428d2.jpg,https://go.centaurlabs.com/problem/1197367,Labeled,"['[{""x"": 0.59604716, ""y"": 0.4975766}, {""x"": 0....",29.0,5.0,"[{""x"": 0.630859375, ""y"": 0.30078125}, {""x"": 0....",2.0,"[[{'x': 0.626953125, 'y': 0.30078125}, {'x': 0...",0.33,0.763632,NaN,NaN


In [11]:
subdural1 = pd.read_csv("Results_Subdural Hemorrhage Detection_2020-11-16_21.35.48.040.csv",error_bad_lines=False)
clean_subdural1 = subdural1[(subdural1['Majority Label'] != '[]') | (subdural1['Correct Label'].notna()) & (subdural1['Correct Label'] != '[[]]')]

C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\3142101893.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  subdural1 = pd.read_csv("Results_Subdural Hemorrhage Detection_2020-11-16_21.35.48.040.csv",error_bad_lines=False)


In [12]:
clean_subdural1.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1198320,ID_0061cb462.jpg,https://go.centaurlabs.com/problem/1198320,Labeled,"['[{""x"": 0.4823523, ""y"": 0.59177}, {""x"": 0.473...",16.0,5.0,"[{""x"": 0.45703125, ""y"": 0.78125}, {""x"": 0.4570...",2.0,"[[{'x': 0.40625, 'y': 0.330078125}, {'x': 0.40...",0.29,0.949092,NaN,NaN
1,1198320,ID_0061cb462.jpg,https://go.centaurlabs.com/problem/1198320,Labeled,"['[{""x"": 0.4823523, ""y"": 0.59177}, {""x"": 0.473...",16.0,5.0,"[{""x"": 0.404296875, ""y"": 0.333984375}, {""x"": 0...",2.0,"[[{'x': 0.40625, 'y': 0.330078125}, {'x': 0.40...",0.29,0.949092,NaN,NaN
2,1198004,ID_0086b5e67.jpg,https://go.centaurlabs.com/problem/1198004,In Progress,"['[{""x"": 0.5224404, ""y"": 0.8304443}, {""x"": 0.5...",15.0,6.0,"[{""x"": 0.521484375, ""y"": 0.83203125}, {""x"": 0....",2.0,NaN,NaN,0.157037,NaN,NaN
3,1198004,ID_0086b5e67.jpg,https://go.centaurlabs.com/problem/1198004,In Progress,"['[{""x"": 0.5224404, ""y"": 0.8304443}, {""x"": 0.5...",15.0,6.0,"[{""x"": 0.478515625, ""y"": 0.83984375}, {""x"": 0....",2.0,NaN,NaN,0.157037,NaN,NaN
4,1198353,ID_00eb6f7cc.jpg,https://go.centaurlabs.com/problem/1198353,Labeled,"['[{""x"": 0.48163298, ""y"": 0.71568763}, {""x"": 0...",15.0,5.0,"[{""x"": 0.447265625, ""y"": 0.73828125}, {""x"": 0....",1.0,"[[{'x': 0.447265625, 'y': 0.73828125}, {'x': 0...",0.00,0.306828,NaN,NaN


In [13]:
subdural2 = pd.read_csv("Results_Subdural Hemorrhage Detection_2020-11-16_21.37.19.745.csv",error_bad_lines=False)
clean_subdural2 = subdural2[(subdural2['Majority Label'] != '[]') | (subdural2['Correct Label'].notna()) & (subdural2['Correct Label'] != '[[]]')]

C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\909399964.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  subdural2 = pd.read_csv("Results_Subdural Hemorrhage Detection_2020-11-16_21.37.19.745.csv",error_bad_lines=False)


In [14]:
clean_subdural2.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1198320,ID_0061cb462.jpg,https://go.centaurlabs.com/problem/1198320,Labeled,"['[{""x"": 0.4823523, ""y"": 0.59177}, {""x"": 0.473...",16.0,5.0,"[{""x"": 0.45703125, ""y"": 0.78125}, {""x"": 0.4570...",2.0,"[[{'x': 0.40625, 'y': 0.330078125}, {'x': 0.40...",0.29,0.949092,NaN,NaN
1,1198320,ID_0061cb462.jpg,https://go.centaurlabs.com/problem/1198320,Labeled,"['[{""x"": 0.4823523, ""y"": 0.59177}, {""x"": 0.473...",16.0,5.0,"[{""x"": 0.404296875, ""y"": 0.333984375}, {""x"": 0...",2.0,"[[{'x': 0.40625, 'y': 0.330078125}, {'x': 0.40...",0.29,0.949092,NaN,NaN
2,1198004,ID_0086b5e67.jpg,https://go.centaurlabs.com/problem/1198004,In Progress,"['[{""x"": 0.5224404, ""y"": 0.8304443}, {""x"": 0.5...",15.0,6.0,"[{""x"": 0.521484375, ""y"": 0.83203125}, {""x"": 0....",2.0,NaN,NaN,0.157037,NaN,NaN
3,1198004,ID_0086b5e67.jpg,https://go.centaurlabs.com/problem/1198004,In Progress,"['[{""x"": 0.5224404, ""y"": 0.8304443}, {""x"": 0.5...",15.0,6.0,"[{""x"": 0.478515625, ""y"": 0.83984375}, {""x"": 0....",2.0,NaN,NaN,0.157037,NaN,NaN
4,1198353,ID_00eb6f7cc.jpg,https://go.centaurlabs.com/problem/1198353,Labeled,"['[{""x"": 0.48163298, ""y"": 0.71568763}, {""x"": 0...",15.0,5.0,"[{""x"": 0.447265625, ""y"": 0.73828125}, {""x"": 0....",1.0,"[[{'x': 0.447265625, 'y': 0.73828125}, {'x': 0...",0.00,0.306828,NaN,NaN


In [15]:
# adding label column to all the dataframes
# ignore the warnings

# mapping of dummy label to class
label_key = {0: "epi", 1: "intra", 2: "multi", 3: "subarachnoid", 4: "subdural1", 5: "subdural2"}

clean_epi["label"] = 0
clean_intra["label"] = 1
clean_multiple["label"] = 2
clean_subarachnoid["label"] = 3
clean_subdural1["label"] = 4
clean_subdural2["label"] = 5

C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\3286737939.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_epi["label"] = 0
C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\3286737939.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_intra["label"] = 1
C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\3286737939.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [26]:
from skimage import io
from PIL import Image
import requests
from io import BytesIO
import ast
#adding images to the dataframe

url = 'https://api.github.com/repos/Red-Nova/MATH4570-Project-Data/git/trees/9d6debaa5593e60451476d5271fc482109052bba'
response = requests.get(url, headers={"Accept": "application/vnd.github.v3+json"})
img_dict_list = response.json().get("tree")
for dict in img_dict_list:
    print(dict.get('path'))
    if dict.get('path') == clean_epi['Origin'][0]: #This is a TEST to confirm that it can match images
        print(f'Match found:{dict.get("path")}')
        response = requests.get(dict.get('url'))
        print(response.json())
        img = Image.open(response.json().get('content'))
        img = np.matrix(img)
        img_list.append(img)
        
# Below is the 
        

# img_list = [] #initialize an list, which will become an np array appended to the picture_and_label dataframe
# for i, f in enumerate(clean_epi["Origin"]):
#     #There is a more modular way to do this based on directory name. If we anticpate more commits to the image directory,
#     #that will become necessary
#     brain_bone_url = 'https://api.github.com/repos/Red-Nova/MATH4570-Project-Data/git/trees/9d6debaa5593e60451476d5271fc482109052bba'
#     brain_url = "https://api.github.com/repos/Red-Nova/MATH4570-Project-Data/git/trees/908d3d1d845a72db02983343455c69a13dca914b"
#     max_contrast_url = "https://api.github.com/repos/Red-Nova/MATH4570-Project-Data/git/trees/7ccffd4eef8537b80b45de3a76683f23753ca55c"
#     subdural_url = "https://api.github.com/repos/Red-Nova/MATH4570-Project-Data/git/trees/0692e92840a05638292dd455b7e76d7a769472a9"
#     response = requests.get(url, headers={"Accept": "application/vnd.github.v3+json"})
#     img = Image.open(BytesIO(response.content))
#     img = np.matrix(img)
#     img_list.append(img)

ID_000edbf38.jpg
ID_00178eb80.jpg
ID_001bb2c00.jpg
ID_0026de01c.jpg
ID_004966e2d.jpg
ID_004c4b319.jpg
Match found:ID_004c4b319.jpg
{'sha': '3898ac77eacdd28a411e3f4f683b916919b84dfa', 'node_id': 'B_kwDOHIm-pNoAKDM4OThhYzc3ZWFjZGQyOGE0MTFlM2Y0ZjY4M2I5MTY5MTliODRkZmE', 'size': 12213, 'url': 'https://api.github.com/repos/Red-Nova/MATH4570-Project-Data/git/blobs/3898ac77eacdd28a411e3f4f683b916919b84dfa', 'content': '/9j/4AAQSkZJRgABAQEAZABkAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsL\nDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/\n2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIy\nMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAIAAgADASIAAhEBAxEB/8QA\nHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUF\nBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkK\nFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1\ndnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXG\nx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEB\nAQEBAQAAAAAAAAECAwQFBgcICQo

FileNotFoundError: [Errno 2] No such file or directory: '/9j/4AAQSkZJRgABAQEAZABkAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsL\nDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/\n2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIy\nMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAIAAgADASIAAhEBAxEB/8QA\nHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUF\nBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkK\nFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1\ndnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXG\nx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEB\nAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAEC\nAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRom\nJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOE\nhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU\n1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD5/ooooAKK\nKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooo\noAKKKKACiiigA

In [25]:
# combining all the dataframes
combined = clean_epi.append(clean_intra)
combined = combined.append(clean_multiple)
combined = combined.append(clean_subarachnoid)
combined = combined.append(clean_subdural2)
combined

C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\3476849609.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = clean_epi.append(clean_intra)
C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\3476849609.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = combined.append(clean_multiple)
C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\3476849609.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = combined.append(clean_subarachnoid)
C:\Users\sahsa\AppData\Local\Temp\ipykernel_8552\3476849609.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = combined.append(clean_subdural2)


,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes,label
0,1196514,ID_004c4b319.jpg,https://go.centaurlabs.com/problem/1196514,Labeled,"['[{""x"": 0.40818718, ""y"": 0.8623607}, {""x"": 0....",19.0,5.0,"[{""x"": 0.537109375, ""y"": 0.89453125}, {""x"": 0....",1.0,"[[{'x': 0.537109375, 'y': 0.89453125}, {'x': 0...",0.00,0.968851,NaN,NaN,0
1,1196682,ID_0237b835f.jpg,https://go.centaurlabs.com/problem/1196682,Ready,"['[{""x"": 0.62959146, ""y"": 0.720186}, {""x"": 0.6...",51.0,28.0,"[{""x"": 0.654296875, ""y"": 0.724609375}, {""x"": 0...",1.0,NaN,NaN,0.063519,NaN,NaN,0
2,1196477,ID_02e2d273d.jpg,https://go.centaurlabs.com/problem/1196477,Labeled,"['[{""x"": 0.35205495, ""y"": 0.34680602}, {""x"": 0...",22.0,6.0,"[{""x"": 0.46484375, ""y"": 0.357421875}, {""x"": 0....",1.0,"[[{'x': 0.46484375, 'y': 0.357421875}, {'x': 0...",0.06,0.710300,NaN,NaN,0
3,1196612,ID_02f37ce84.jpg,https://go.centaurlabs.com/problem/1196612,Ready,"['[{""x"": 0.3680166, ""y"": 0.48387372}, {""x"": 0....",51.0,29.0,"[{""x"": 0.431640625, ""y"": 0.607421875}, {""x"": 0...",2.0,NaN,NaN,0.264134,NaN,NaN,0
4,1196612,ID_02f37ce84.jpg,https://go.centaurlabs.com/problem/1196612,Ready,"['[{""x"": 0.3680166, ""y"": 0.48387372}, {""x"": 0....",51.0,29.0,"[{""x"": 0.353515625, ""y"": 0.59765625}, {""x"": 0....",2.0,NaN,NaN,0.264134,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,1198392,ID_ff1e93267.jpg,https://go.centaurlabs.com/problem/1198392,Labeled,"['[{""x"": 0.25159726, ""y"": 0.24648584}, {""x"": 0...",19.0,5.0,"[{""x"": 0.68359375, ""y"": 0.42578125}, {""x"": 0.6...",3.0,"[[{'x': 0.685546875, 'y': 0.43359375}, {'x': 0...",0.48,0.607623,NaN,NaN,5
739,1198392,ID_ff1e93267.jpg,https://go.centaurlabs.com/problem/1198392,Labeled,"['[{""x"": 0.25159726, ""y"": 0.24648584}, {""x"": 0...",19.0,5.0,"[{""x"": 0.630859375, ""y"": 0.24609375}, {""x"": 0....",3.0,"[[{'x': 0.685546875, 'y': 0.43359375}, {'x': 0...",0.48,0.607623,NaN,NaN,5
740,1198414,ID_ff2f3dbb7.jpg,https://go.centaurlabs.com/problem/1198414,Labeled,"['[{""x"": 0.70854944, ""y"": 0.6837957}, {""x"": 0....",14.0,6.0,"[{""x"": 0.70703125, ""y"": 0.67578125}, {""x"": 0.7...",1.0,"[[{'x': 0.70703125, 'y': 0.67578125}, {'x': 0....",0.15,0.153092,NaN,NaN,5
741,1198334,ID_04ab5246e.jpg,https://go.centaurlabs.com/problem/1198334,Labeled,"['[{""x"": 0.56073856, ""y"": 0.6705539}, {""x"": 0....",13.0,6.0,[],0.0,"[[{'x': 0.560546875, 'y': 0.669921875}, {'x': ...",0.33,0.833333,NaN,NaN,5


In [ ]:
# removing flagged images
with open("flagged.txt", "r") as f:
    lines = f.readlines()
flagged = []
for l in lines:
    flagged.append(l.strip())
combined = combined.drop(combined[combined["Origin"].isin(flagged)].index)

In [ ]:
# loading images
picture_and_label = combined[["Origin", "label"]]

from skimage import io
from PIL import Image
import requests
from io import BytesIO

url = 'https://api.github.com/repos/Red-Nova/MATH4570-Project-Data/git/trees/1d54bc82a5135e9404ec03ceee5040bc8a8793c2'
response = requests.get(url, headers={"Accept": "application/vnd.github.v3+json"})
print(response.status_code)
print(response.content)
# img = Image.open(BytesIO(response.content))
# img = np.matrix(img)
# img_list.append(img)

# import glob
# from pathlib import Path

# path = Path("../4570-Project/")
# images = []
# print(path)
# for file in path.iterdir():
    # print(file)
    # if file in picture_and_label["Origin"]:
        # img = cv2.imread(file)
    # if img is not None:
        # images.append(img)
# images


# img_list = [] #initialize an list, which will become an np array appended to the picture_and_label datafranme
# for i, f in enumerate(picture_and_label["Origin"]):
# #     url = 'https://api.github.com/repos/Red-Nova/MATH4570-Project-Data/tree/main/BrainCT/epidural/epidural/brain_bone_window/0592fec14a798943460120b330fb628ee382bb55'
# #     response = requests.get(url)
# #     img = Image.open(BytesIO(response.content))
#     img = io.imread(f, plugin='matplotlib')
#     img = np.matrix(img)
#     img_list.append(img)
#     # it should be ok to skip this line, because the dataframe will form it correctly anyway without explicit indexing
# #     picture_and_label.loc(i, "img") = img 
# picture_and_label = pd.concat(picture_and_label, np.ndarray(img_list), axis=1)

In [ ]:
#some of the labels are nans but py thinks of them as strings(sometimes)
def isNaN(string):
    return string != string